In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt

Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
%matplotlib inline

In [25]:
# regression: 1 lable range 0-1
train_data = pickle.load( open( "../train_data_NR.p", "rb" ) )
train_labels = pickle.load( open( "../train_labels_NR.p", "rb" ) )
test_data = pickle.load( open( "../test_data_NR.p", "rb" ) )
test_labels = pickle.load( open( "../test_labels_NR.p", "rb" ) )
val_data = pickle.load( open( "../val_data_NR.p", "rb" ) )
val_labels = pickle.load( open( "../val_labels_NR.p", "rb" ) )


In [19]:
# # get subset of 1000 samples
subset_train_data = train_data[0:100]
subset_train_labels = train_labels[0:100]

In [11]:
# classification:  21 labels one hot, 4 features
train_data = pickle.load( open( "../train_data2.p", "rb" ) )
train_labels = pickle.load( open( "../train_labels2.p", "rb" ) )
test_data = pickle.load( open( "../test_data2.p", "rb" ) )
test_labels = pickle.load( open( "../test_labels2.p", "rb" ) )
val_data = pickle.load( open( "../val_data2.p", "rb" ) )
val_labels = pickle.load( open( "../val_labels2.p", "rb" ) )


In [26]:
# pickle data
d_data = pickle.load(open("../d_data.p", "rb"))
d_labels = pickle.load(open("../d_labels_r.p", "rb"))

sub_d_data = d_data[:7000]
sub_d_labels = d_labels[:7000]

In [27]:
G = "NG" # or "NG"
forget_b = 1
val_step = 10000000
# val_step = 100

# Training Parameters
learning_rate = 0.0001
# training_steps = 1000000 #10000000 #1000000
training_steps = 10000000
# training_steps = 100
batch_size = 1 # 128
display_step = 100000
# display_step = 10

# Network Parameters
num_input = 4    # 28 # MNIST data input (img shape: 28*28)
timesteps = 23   # 28 # timesteps
num_hidden = 64  # 10 # hidden layer num of features
num_classes = 1  # 10 # MNIST total classes (0-9 digits)

subset=0

In [15]:
# get subset of 10 samples
if(subset>0) :
    subset_train_data = train_data[0:subset]
    subset_train_labels = train_labels[0:subset]

In [28]:
#this resets the graph - re-run tensor-flow specific things after it
tf.reset_default_graph() 

# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])

# Define weights
weights = {
    # Hidden layer weights => 2*n_hidden because of forward + backward cells
    'out': tf.get_variable("my_int_variable", [2*num_hidden, num_classes], 
  initializer=tf.glorot_uniform_initializer(seed = 23))
#     tf.Variable(tf.random_normal([2*num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

def BiRNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, num_input)
    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, num_input)
    x = tf.unstack(x, timesteps, 1)

    # Define lstm cells with tensorflow
#     lstm_fw_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=forget_b)    # Forward direction cell
#     lstm_bw_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=forget_b)    # Backward direction cell

        ################ try relu activation instead
    # Forward direction cell
    lstm_fw_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=forget_b, activation=tf.nn.relu)
    # Backward direction cell
    lstm_bw_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=forget_b, activation=tf.nn.relu)

    try:                            # Get lstm cell output
        outputs, _, _ = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x,
                                              dtype=tf.float32)
    except Exception:               # Old TensorFlow version only returns outputs not states
        outputs = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x,
                                        dtype=tf.float32)

    # Linear activation, using rnn inner loop last output then sigmoid result to get output in range [0,1]

    # no sigmoid
    #return tf.matmul(outputs[-1], weights['out']) + biases['out']
    
    # sigmoid
    return tf.nn.sigmoid(tf.matmul(outputs[-1], weights['out']) + biases['out'])

prediction = BiRNN(X, weights, biases)

# Define loss and optimizer for classification:
#loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=Y))
# for regression:
loss_op = tf.reduce_mean(tf.losses.mean_squared_error(labels=Y, predictions=prediction))

# loss_op = tf.reduce_mean(tf.square(Y - prediction))            # try reduce mean sq without using built in mse fn

# optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)  # switch to adam optimizer
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
# prediction = tf.nn.softmax(prediction)
# prediction = tf.argmax(tf.nn.softmax(prediction), 1)

# this is for classification
# correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
# correct_pred = tf.equal(prediction, tf.argmax(Y, 1))
# accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

#for regression
accuracy = tf.reduce_mean(tf.losses.mean_squared_error(Y, prediction))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [29]:
def get_batch(batch_size, x, y):
    i = np.random.randint(0,x.shape[0], size=(batch_size))
    return np.array(x[i]), np.array(y[i])

In [ ]:
# train and val on same data
# train_data = subset_train_data
# train_labels = subset_train_labels
# val_data = subset_train_data
# val_labels = subset_train_labels

with tf.Session() as sess:               # Start training
    sess.run(init)                       # Run the initializer
    
    # things to save during display step + val step
    train_losses = []
    val_losses = []
    
    for step in range(1, training_steps+1):
        batch_x, batch_y = get_batch(batch_size, train_data, train_labels)
        
        # Reshape data to get 28 seq of 28 elements        
        # comment out with 3 or 21 labels + no-gene.... need for regression
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        batch_y = batch_y.reshape((batch_size, num_classes))
        
        # Run optimization op (backprop) in every training step
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        
        # in display step, clac train_losses
        if step % display_step == 0 or step == 1:
            t_loss = sess.run(loss_op, feed_dict={X: batch_x, Y: batch_y})
            train_losses.append(t_loss)
        
        # in val step, calc val_losses
        if step % val_step == 0 or step == 1:
            vl_temp = []
            for i in range(val_labels.shape[0]/batch_size):
#               val_labels = val_labels.reshape((val_labels.shape[0], num_classes)) # < don't need this??
                x = val_data[i*batch_size: i*batch_size + batch_size].reshape((batch_size,timesteps, num_input))
                y = val_labels[i*batch_size:i*batch_size + batch_size].reshape((batch_size,num_classes))
                vl_temp.append(sess.run(loss_op, feed_dict={X: x, Y:y}))
            v_loss = np.mean(vl_temp)
            val_losses.append(v_loss)

            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(t_loss) + ", Validation Loss= " + "{:.5f}".format(v_loss))
        
    print("Optimization Finished!")

    
    
#     # Calculate accuracy and predictions for validation set
#     val_preds = []
#     val_accs_final = []
#     val_losses_final = []

#     for i in range(val_labels.shape[0]/batch_size):
# #       val_labels = val_labels.reshape((val_labels.shape[0], num_classes)) # < don't need this??
#         x = val_data[i*batch_size: i*batch_size + batch_size].reshape((batch_size,timesteps, num_input))
#         y = val_labels[i*batch_size:i*batch_size + batch_size].reshape((batch_size,num_classes))
#         val_preds.append(sess.run(prediction, feed_dict={X: x}))      
#         val_accs_final.append(sess.run(accuracy, feed_dict={X: x, Y:y}))
#         val_losses_final.append(sess.run(loss_op, feed_dict={X: x, Y:y}))
    
#     val_acc_f = np.mean(val_accs_final)
#     val_loss_f = np.mean(val_losses_final)
    
#     print("Final Validation Accuracy = " + "{:.4f}".format(val_acc_f))
#     print("Final Validation Loss = " + "{:.4f}".format(val_loss_f))
    
    
    #######################################################################################################
    # Calculate accuracy and predictions for validation set
    val_preds = []
    val_accs_final = []
    val_losses_final = []

    for i in range(val_labels.shape[0]/batch_size):
#         val_labels = val_labels.reshape((val_labels.shape[0], num_classes)) # < don't need this??
        x = val_data[i*batch_size: i*batch_size + batch_size].reshape((batch_size,timesteps, num_input))
        y = val_labels[i*batch_size:i*batch_size + batch_size].reshape((batch_size,num_classes))
        val_preds.append(sess.run(prediction, feed_dict={X: x}))      
        val_accs_final.append(sess.run(accuracy, feed_dict={X: x, Y:y}))
        val_losses_final.append(sess.run(loss_op, feed_dict={X: x, Y:y}))
    
    val_acc_f = np.mean(val_accs_final)
    val_loss_f = np.mean(val_losses_final)
    
    print("Final Validtion Accuracy = " + "{:.4f}".format(val_acc_f))
    print("Final Validation Loss = " + "{:.4f}".format(val_loss_f))
    
    
    
    
    #######################################################################################################
    # Calculate accuracy and predictions for testing set
    test_preds = []
    test_accs_final = []
    test_losses_final = []

    for i in range(test_labels.shape[0]/batch_size):
#         val_labels = val_labels.reshape((val_labels.shape[0], num_classes)) # < don't need this??
        x = test_data[i*batch_size: i*batch_size + batch_size].reshape((batch_size,timesteps, num_input))
        y = test_labels[i*batch_size:i*batch_size + batch_size].reshape((batch_size,num_classes))
        test_preds.append(sess.run(prediction, feed_dict={X: x}))      
        test_accs_final.append(sess.run(accuracy, feed_dict={X: x, Y:y}))
        test_losses_final.append(sess.run(loss_op, feed_dict={X: x, Y:y}))
    
    test_acc_f = np.mean(test_accs_final)
    test_loss_f = np.mean(test_losses_final)
    
    print("Final Test Accuracy = " + "{:.4f}".format(test_acc_f))
    print("Final Test Loss = " + "{:.4f}".format(test_loss_f))
    
    #######################################################################################################
    # Calculate accuracy and predictions for doench data
    d_preds = []
    d_accs_final = []
    d_losses_final = []
    d_data = sub_d_data
    d_labels = sub_d_labels

    for i in range(d_labels.shape[0]/batch_size):
#         val_labels = val_labels.reshape((val_labels.shape[0], num_classes)) # < don't need this??
        x = d_data[i*batch_size: i*batch_size + batch_size].reshape((batch_size,timesteps, num_input))
        y = d_labels[i*batch_size:i*batch_size + batch_size].reshape((batch_size,num_classes))
        d_preds.append(sess.run(prediction, feed_dict={X: x}))      
        d_accs_final.append(sess.run(accuracy, feed_dict={X: x, Y:y}))
        d_losses_final.append(sess.run(loss_op, feed_dict={X: x, Y:y}))
    
    d_acc_f = np.mean(d_accs_final)
    d_loss_f = np.mean(d_losses_final)
    
    print("Final Doench Accuracy = " + "{:.4f}".format(d_acc_f))
    print("Final Doench Loss = " + "{:.4f}".format(d_loss_f))

Step 1, Minibatch Loss= 0.0833, Validation Loss= 0.06163


In [ ]:
# plot for val
preds = [v[0] for v in val_preds]
preds_1d = np.array(preds).reshape(len(preds),)
vals_1d = val_labels.reshape(val_labels.shape[0],)

plt.scatter(vals_1d, preds_1d, alpha=0.2)

z = np.polyfit(vals_1d, preds_1d, 1)
p = np.poly1d(z)
plt.plot(vals_1d, p(vals_1d), "r")

plt.title("Predictions for validation set: regression, lr = 0.0001, iter = 10000000, relu")
# plt.title("Predictions for 100 training set: regression, lr = 0.001, iter = 100000, relu")


plt.ylabel("predicted effect")
plt.xlabel("actual effect")
plt.show()
plt.savefig("val_preds_r_relu")

In [ ]:
import scipy.stats as st
st.spearmanr(preds_1d, vals_1d)


In [ ]:
# plot for test
tpreds = [v[0] for v in test_preds]
tpreds_1d = np.array(tpreds).reshape(len(tpreds),)
tlabs_1d = test_labels.reshape(test_labels.shape[0],)

plt.scatter(tlabs_1d, tpreds_1d, alpha=0.2)

z = np.polyfit(tlabs_1d, tpreds_1d, 1)
p = np.poly1d(z)
plt.plot(tlabs_1d, p(tlabs_1d), "r")

plt.title("Predictions for testing set: regression, lr = 0.0001, iter = 10000000, relu")
plt.ylabel("predicted effect")
plt.xlabel("actual effect")
plt.show()
plt.savefig("test_preds_r_relu")

In [ ]:
st.spearmanr(tpreds_1d, tlabs_1d)

In [ ]:
# plot for doench
dpreds = [v[0] for v in d_preds]
dpreds_1d = np.array(dpreds).reshape(len(dpreds),)
dlabs_1d = d_labels.reshape(d_labels.shape[0],)

plt.scatter(dlabs_1d, dpreds_1d, alpha=0.2)

z = np.polyfit(dlabs_1d, dpreds_1d, 1)
p = np.poly1d(z)
plt.plot(dlabs_1d, p(dlabs_1d), "r")

plt.title("Predictions for doench data: regression, lr = 0.0001, iter = 10000000, relu")
plt.ylabel("predicted effect")
plt.xlabel("actual effect")
plt.show()
plt.savefig("doench_preds_r_relu")

In [ ]:
st.spearmanr(dpreds_1d, dlabs_1d)

In [ ]:
# plot training loss over time
plt.scatter(np.arange(0, training_steps+display_step, display_step), train_losses)
plt.xlabel("training iteration")
plt.ylabel("training loss")
plt.title("MSE loss for training data: regression, lr = 0.0001, iter = 10000000, relu")
plt.savefig("train_loss_r_relu")

In [ ]:
####### saving results summary for each run

In [ ]:
# 1. overall_results
# columns = gene(G/NG) , num_classes(1,3,21), num_hidden(8,32,128), lr(0.1,0.01,0.001,0.0001), 
    # forget(0,0.5,1), weight(0,1,2), bias(0.01,0.001), train_loss, val_loss, train_acc, val_acc

output_file = open("../r_results_relu_1e7.csv", "a")

if(subset>0) :
#output_lst = [G, num_classes, num_hidden, learning_rate, forget_b, weight_mean, bias_mean, train_losses[-1], val_losses[-1], val_acc]
    output_lst = [G, num_classes, num_hidden, learning_rate, forget_b, train_losses[-1], val_loss_f, val_acc_f, "subset", subset]
else :
    output_lst = [G, num_classes, num_hidden, learning_rate, forget_b, train_losses[-1], val_loss_f, val_acc_f]
    
#output_lst = [G, num_classes, num_hidden, learning_rate, forget_b, weight_mean, bias_mean, train_losses[-1], val_loss_f, val_acc_f]
output_file.writelines(",".join([str(i) for i in output_lst])+"\n")
output_file.close()


In [ ]:
# if(subset>0) :
#     fname = ["subset", subset, G, num_classes, num_hidden, learning_rate, forget_b]
# else :
#     fname = [G, num_classes, num_hidden, learning_rate, forget_b]

In [ ]:
fname

In [ ]:
# 2a. prediction and true labels for val
# (preds_1d, vals_1d) instead of preds, vl
# (dpreds_1d, dlabs_1d), (tpreds_1d, tlabs_1d)

pv_fname = "../rfinal_val_preds_relu1e7" + "_".join([str(i) for i in output_lst]) + ".csv"
pred_vlabels = np.vstack((np.array(preds_1d), vals_1d)).T
np.savetxt(pv_fname, pred_vlabels, delimiter=",")

# 2b. prediction and true labels for test
pt_fname = "../rfinal_test_preds_relu1e7" + "_".join([str(i) for i in output_lst]) + ".csv"
pred_tlabels = np.vstack((np.array(tpreds_1d), tlabs_1d)).T
np.savetxt(pt_fname, pred_tlabels, delimiter=",")

# 2c. prediction and true labels for doench
pd_fname = "../rfinal_d_preds_relu1e7" + "_".join([str(i) for i in output_lst]) + ".csv"
pred_dlabels = np.vstack((np.array(dpreds_1d), dlabs_1d)).T
np.savetxt(pd_fname, pred_dlabels, delimiter=",")

In [ ]:
# 3. training loss over time
tl_fname = "../rfinal_train_loss_relu1e7" + "_".join([str(i) for i in output_lst]) + ".csv"
train_loss_time = np.vstack((np.arange(0, training_steps+display_step, display_step), train_losses)).T
np.savetxt(tl_fname, train_loss_time, delimiter=",")

In [ ]:
# 4. validation loss over time
vl_fname = "../rfinal_val_loss_relu1e7" + "_".join([str(i) for i in output_lst]) + ".csv"
val_loss_time = np.vstack((np.arange(0, training_steps+val_step, val_step), val_losses)).T
np.savetxt(vl_fname, val_loss_time, delimiter=",")